# machine_learning_with_python 
### Regression project
##### For this project, we used the King County Housing dataset from Kaggle, which includes homes sold between May 2014 and May 2015, 21 columns, and over 21,000 entries. We intended to use the features of this dataset to estimate price in dollars, but in this experiment, we were not looking for the best formula for this purpose, but rather to show the way of thinking and the path to the model, as well as different regression methods, and also to test some of the features of Python in this context.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np     
import scipy.stats as stats            # Quantile Quantile plots
from scipy.optimize import curve_fit   # For fit data in Non Linear Regression
import seaborn as sns  #
from sklearn import linear_model       # import linear models from sklearn. Read https://scikit-learn.org/stable/ for learn more
from sklearn.metrics import r2_score   # for calculate R2

%matplotlib inline

### Reading the data in
#### The data is stored in the file housePrice.csv.The building specifications include the following:
#### - id : Unique identifier for a house
#### - date : Date house was sold (2014,2015)
#### - price : Sale price (prediction target)
#### - bedrooms : Number of bedrooms
#### - bathrooms : Number of bathrooms
#### - sqft_living : Square footage of living space in the home
#### - sqft_lot : Square footage of the lot
#### - floors : Number of floors (levels) in house
#### - waterfront : Whether the house is on a waterfront
#### - view : Quality of view from house
#### - condition : How good the overall condition of the house is. Related to maintenance of house. See the King County      Assessor Website for further explanation of each condition code
#### - grade : Overall grade of the house. Related to the construction and design of the house. See the King County Assessor Website for further explanation of each building grade code
#### - sqft_above : Square footage of house apart from basement
#### - sqft_basement : Square footage of the basement
#### - yr_built : Year when house was built
#### - yr_renovated : Year when house was renovated
#### - zipcode : ZIP Code used by the United States Postal Service
#### - lat : Latitude coordinate
#### - long : Longitude coordinate
#### - sqft_living15 : The square footage of interior housing living space for the nearest 15 neighbors
#### - sqft_lot15 : The square footage of the land lots of the nearest 15 neighbors

In [ ]:
df = pd.read_csv(r".\data\kc_house_data.csv")   # Read cvs from  \data folder and write in df DataFrame
df.head()

### Data Preparation
##### - We dropped yr_renovated and waterfront columns from the dataset because they are missing more than 10% of their values. 
##### - We also dropped id and date columns because their not needed. 
##### - We needed to remove null values.


In [ ]:
df.drop('yr_renovated', axis=1, inplace=True)  # Drop column
df.drop('waterfront', axis=1, inplace=True)    # Drop column
df.drop('id', axis=1, inplace=True)            # Drop column
df.drop('date', axis=1, inplace=True)          # Drop column
df = df.dropna()                               # remove null values
df['view'] = df['view'].apply(str)             # Convert to String
df['condition'] = df['condition'].apply(str)   # Convert to String
df['grade'] = df['grade'].apply(str)           # Convert to String
df.head()

##### We also dropped duplicate rows

In [ ]:
df_cleaned = df.drop_duplicates()    #remove duplicate rows
df_cleaned.head()

### Data Analysis
##### Here we were trying to analyze the data using histograms of different variables. As we could see in the histograms below, some of our variables were not normally distributed and the scales were not consistent. Our independent variable “price” had a significant positive skewness that we needed to adjust before modeling. 

In [ ]:

plt.figure(figsize=(15, 15))
plt.subplot(4, 3, 1)
plt.hist(df['price'],edgecolor="red")
plt.title('price')
plt.subplot(4, 3, 2)
plt.hist(df['bedrooms'],edgecolor="red")
plt.title('bedrooms')
plt.subplot(4, 3, 3)
plt.hist(df['bathrooms'],edgecolor="red")
plt.title('bathrooms')
plt.subplot(4, 3, 4)
plt.hist(df['sqft_living'],edgecolor="red")
plt.title('sqft_living')
plt.subplot(4, 3, 5)
plt.hist(df['sqft_lot'],edgecolor="red")
plt.title('sqft_lot')
plt.subplot(4, 3, 6)
plt.hist(df['floors'],edgecolor="red")
plt.title('floors')
plt.subplot(4, 3, 7)
plt.hist(df['sqft_above'],edgecolor="red")
plt.title('sqft_above')
plt.subplot(4, 3, 8)
plt.hist(df['sqft_basement'],edgecolor="red")
plt.title('sqft_basement')
plt.subplot(4, 3, 9)
plt.hist(df['yr_built'],edgecolor="red")
plt.title('yr_built')
plt.subplot(4, 3, 10)
plt.hist(df['sqft_living15'],edgecolor="red")
plt.title('sqft_living15')
plt.subplot(4, 3, 11)
plt.hist(df['sqft_lot15'],edgecolor="red")
plt.title('sqft_lot15')
plt.show()


##### Our categorical variables (view, status, and score), as we could see from the corresponding histograms, had a strong linear relationship with price.

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
df['view'].value_counts().plot(kind='bar')
plt.subplot(1, 3, 2)
df['condition'].value_counts().plot(kind='bar')
plt.subplot(1, 3, 3)
df['grade'].value_counts().plot(kind='bar')
plt.show()


#### Dummy Encoding
##### In this model, it seemed that categorical variables should be coded as dummy variables. This meant that each category should be converted into a new column and assigned the number 1 or 0, depending on the original column.
##### We used the pandas function get_dummies.

In [ ]:

df = pd.get_dummies(df)
display(df)


##### We created a heatmap of the correlations to see which variables had the highest correlation with our target variable, price. This was also done to check for multicollinearity of the features. This means that we will only use one of the variables (other than price) that has a high correlation in the modeling.

In [ ]:

plt.figure(figsize=(40, 40))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
#creating list of features and correlations 
corr = df.corr()
features = []
correlations = []
for idx, correlation in corr['price'].items():
    if correlation >= .30 and idx != 'price':
        features.append(idx)
        correlations.append(correlation)
corr_price_df = pd.DataFrame({'Correlations':correlations, 'Features': features}).sort_values(by=['Correlations'], ascending=False)

#checking for multicollinearity
MC_Features = []
MC_Corr = []
def check_MC(feature):
    for idx, correlation in corr[feature].T.items():
        if correlation >= .74 and idx != feature:
            MC_Features.append([feature, idx])
            MC_Corr.append(correlation)
            
for feature in corr:
    check_MC(feature)
MC_df = pd.DataFrame({'Correlations':MC_Corr, 'Features': MC_Features}).sort_values(by=['Correlations'], ascending=False)

#printing variable
print('Correlations with Price')
display(corr_price_df)
print('Multicollinear Features')
display(MC_df)

### We also had to check that the variables used for modeling met the assumptions of linear regression.
#### These assumptions are:
#### -The variable must have a linear relationship with the price.
#### -The variable should be homoscedastic meaning that an equal amount of variance should be seen around the regression line.
#### -There should be a normal distribution.
##### To examine homoscedastiscity in the most correlated variables and normal distribution, we can use histograms (above) and pairwise plots (below).

In [ ]:
# using a pairplot 
sns.set()
cols = ['price','sqft_living', 'sqft_above', 'sqft_living15', 'bedrooms']     # Columns with most Correlations with Price
sns.pairplot(df[cols], height = 2.5)
plt.show();

##### The variables Sqft_living, sqft_living15, and sqft_above all showed a funnel-like shape in the pairwise plot, so they did not satisfy the assumptions of homogeneity of variance, and we had to transform these variables before modeling. The QQ charts above also show that the price column and the other columns are not normal. So we have reached this conclusion in three different experiments and charts.
#####This columns needed to be log/sqrt/cbrt transformed so that they could meet the assumptions for linear regression. After that, we saw if the three different experiments and charts show better conditions or not.

In [ ]:
df['log_price'] = np.log2(df['price'])   # Use price's log
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_price'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('Log(price)')
plt.subplot(1, 3, 2)
plt.hist(df['log_price'],edgecolor="red")
plt.title("Log(price)")
plt.show()

In [ ]:
df['log_bedrooms'] = np.cbrt(df['bedrooms'])   # Use bedrooms's cube-root
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_bedrooms'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('cbrt(bedrooms)')
plt.subplot(1, 3, 2)
plt.hist(df['log_bedrooms'],edgecolor="red")
plt.title("cbrt(bedrooms)")
plt.show()

In [ ]:
df['log_bathrooms'] = np.cbrt(df['bathrooms'])  # Use bedrooms's cube-root
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_bathrooms'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('cbrt(log_bathrooms)')
plt.subplot(1, 3, 2)
plt.hist(df['log_bathrooms'],edgecolor="red")
plt.title("cbrt(log_bathrooms)")
plt.show()

In [ ]:
df['log_sqft_living'] = np.log2(df['sqft_living'])  # Use sqft_living's log
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_sqft_living'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('Log(sqft_living)')
plt.subplot(1, 3, 2)
plt.hist(df['log_sqft_living'],edgecolor="red")
plt.title("Log(sqft_living)")
plt.show()


In [ ]:
df['log_sqft_lot'] = np.log2(df['sqft_lot'])  # Use sqft_living's log
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_sqft_lot'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('Log(sqft_lot)')
plt.subplot(1, 3, 2)
plt.hist(df['log_sqft_lot'],edgecolor="red")
plt.title("Log(sqft_lot)")
plt.show()


In [ ]:
df['log_sqft_above'] = np.log2(df['sqft_above'])  # Use sqft_above's log
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_sqft_above'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('Log(sqft_above)')
plt.subplot(1, 3, 2)
plt.hist(df['log_sqft_above'],edgecolor="red")
plt.title("Log(sqft_above)")
plt.show()


In [ ]:
df['log_sqft_living15'] = np.log2(df['sqft_living15'])  # Use sqft_living15's log
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_sqft_living15'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('Log(sqft_living15)')
plt.subplot(1, 3, 2)
plt.hist(df['log_sqft_living15'],edgecolor="red")
plt.title("Log(sqft_living15)")
plt.show()



In [ ]:
df['log_sqft_lot15'] = np.log2(df['sqft_lot15'])  # Use sqft_lot15's log
plt.figure(figsize=(15, 2.5))
plt.subplot(1, 3, 1)
stats.probplot(df['log_sqft_lot15'], dist="norm", plot=plt)     # Draw Q-Q plot
plt.title('Log(sqft_lot15)')
plt.subplot(1, 3, 2)
plt.hist(df['log_sqft_lot15'],edgecolor="red")
plt.title("Log(sqft_lot15)")
plt.show()

In [ ]:
sns.set()
cols = ['log_price','log_sqft_living', 'log_sqft_above', 'log_sqft_living15', 'log_bedrooms']
sns.pairplot(df[cols], height = 2.5)
plt.show();

##### Compared to our previous pairplot, there was no longer a funnel like shape with our transformed variables. This indicated that assumptions for linear regression are met and we could use these variables for modeling.
##### Next, we run some different regression methods on the prepared data.
### Simple Linear Regression
##### We tested simple linear regression for columns log_sqft_living.
##### We randomly separated 80% of the data and obtain a model from it, then we tested our model with the remaining 20%, and finally we checked the result using the Residual sum of squares (MSE).

In [ ]:
# mask for Area
msk = np.random.rand(len(df)) < 0.8
train = df[msk]     # mask for 80%
test = df[~msk]     #mask for 20%

In [ ]:
regr = linear_model.LinearRegression()        # linear model -- Regression
train_x = np.asanyarray(train[['log_sqft_living']])      # data to x array
train_y = np.asanyarray(train[['log_price']])    # data to y array
regr.fit (train_x, train_y)                         # Regression tetas Calculation
# The coefficients
print ('Coefficients: ', regr.coef_)     # teta0
print ('Intercept: ',regr.intercept_)    # teta1

plt.scatter(train.log_sqft_living, train.log_price, color='blue') # scatter from train grade and log_price blue color 
plt.plot(train_x, regr.coef_[0][0]*train_x + regr.intercept_[0], '-r')  # draw a line teta1x + teta0
plt.xlabel("log_sqft_living")
plt.ylabel("log_price")
plt.show()


In [ ]:
test_x = np.asanyarray(test[['log_sqft_living']])         # Test array x
test_y = np.asanyarray(test[['log_price']])       # Test array y
test_y_ = regr.predict(test_x)                       # predicted y 

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))     
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y , test_y_) )     # 

##### It can be seen that the results of Simple Linear Regression are between 45 and less than 50.
### Multiple Linear Regression
##### In this section, we performed two experiments with different columns and examine the results.

In [ ]:
# First experiment
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['grade_13','log_sqft_living','log_sqft_above']]) # x1,x2,x3
y = np.asanyarray(train[['log_price']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)   # teta matrix
print ('Intercept:', regr.intercept_)  # teta0

In [ ]:
x = np.asanyarray(test[['grade_13','log_sqft_living','log_sqft_above']]) # xs  3 x with high correlation
y = np.asanyarray(test[['log_price']])    # ys
y_hat= regr.predict(x)            # predict ys

print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))      #

In [ ]:
# Seconde experiment
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['log_sqft_living15','log_sqft_living', 'log_bedrooms', 'zipcode', 'view_0', 'view_4','grade_7','grade_10', 'grade_11']]) # x1,x2,x3
y = np.asanyarray(train[['log_price']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)   # teta matrix
print ('Intercept:', regr.intercept_)  # teta0

In [ ]:
x = np.asanyarray(test[['log_sqft_living15','log_sqft_living', 'log_bedrooms', 'zipcode', 'view_0', 'view_4','grade_7','grade_10', 'grade_11']]) # xs  3 x with low correlation
y = np.asanyarray(test[['log_price']])    # ys
y_hat= regr.predict(x)            # predict ys

print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))      # 

### Non Linear Regression
##### To show how nonlinear regression works, we assumed that the histogram of log_sqft_living is close to x**2 and tried to fit this plot to our data. However, we did not get a suitable R-squared result.

In [ ]:
def sigmoid(x, Beta_1, Beta_2):      # The function with which we will fit the data
     y = Beta_1 + Beta_2 * np.power(x,2)
     return y

popt, pcov = curve_fit(sigmoid, train['log_sqft_living'], train['log_price'])
#print the final parameters
print(" beta_1 = %f, beta_2 = %f" % (popt[0], popt[1]))



In [ ]:

x=test['log_sqft_living']
plt.figure(figsize=(8,5))
y = sigmoid(x, *popt)
plt.plot(train['log_sqft_living'], train['log_price'], 'ro', label='data')
plt.plot(x,y, linewidth=3.0, label='fit')
plt.legend(loc='best')
plt.ylabel('log_price')
plt.xlabel('log_sqft_living')
plt.show()

##### RMSE and R-squared calculation

In [ ]:
x = np.asanyarray(test[['log_sqft_living']]) # xs 
y = np.asanyarray(test[['log_price']])    # ys

y_hat = sigmoid(test['log_sqft_living'], *popt)

absError = y_hat.to_numpy() - y

SE = np.square(absError) # squared errors
MSE = np.mean(SE) # mean squared errors
RMSE = np.sqrt(MSE) # Root Mean Squared Error, RMSE

Rsquared = 1.0 - (np.var(absError) / np.var(y))
print('RMSE:', RMSE)
print('R-squared:', Rsquared)
